# **Ekstraksi Kata Menggunakan Textrank**

## Deployment

[Streamlit Deployment](https://ekstraksi-kata-kunci.streamlit.app/)

## Install & Importing Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import numpy as np
import nltk
import re
import joblib
import itertools
import networkx as nx



nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PPW/PPW/Resource/Data Berita CNN.csv')
df

,Url,Judul,Tanggal,Artikel,Category
0,https://www.cnnindonesia.com/teknologi/2023110...,"Bakal Jadi Saingan Galaxy Fold 5, Apa Keunggul...","Rabu, 08 Nov 2023 18:31 WIB","Pabrikan smartphoneasal China, Oppo segera mer...",Teknologi
1,https://www.cnnindonesia.com/teknologi/2023110...,"Usai Dinyatakan Langgar Etik, Anwar Usman Tren...","Rabu, 08 Nov 2023 17:01 WIB",Sosok mantan Ketua Mahkamah Konstitusi (MK) An...,Teknologi
2,https://www.cnnindonesia.com/teknologi/2023110...,"Meluncur Tahun Depan, Berikut Daftar Hp Xiaomi...","Rabu, 08 Nov 2023 14:04 WIB",Tampilan user interface (UI) atau antarmuka ba...,Teknologi
3,https://www.cnnindonesia.com/teknologi/2023110...,Ramai-ramai Pengguna Uninstall Aplikasi Pemblo...,"Rabu, 08 Nov 2023 12:47 WIB",Ribuan pengguna ramai-ramai menghapus fitur pe...,Teknologi
4,https://www.cnnindonesia.com/teknologi/2023110...,Lubang Ozon 2023 Masuk 12 Besar Sepanjang Seja...,"Rabu, 08 Nov 2023 11:17 WIB",Lubang ozonAntartika tahun 2023 mencapai ukura...,Teknologi
...,...,...,...,...,...
445,https://www.cnnindonesia.com/olahraga/20231103...,"Kericuhan Pecah Usai Laga PSS vs Bali United, ...","Jumat, 03 Nov 2023 21:27 WIB",Kericuhan pecah usai laga PSS Sleman vs Bali U...,Olahraga
446,https://www.cnnindonesia.com/olahraga/20231103...,"MU Dapat Kabar Buruk, Casemiro Harus Menepi ka...","Jumat, 03 Nov 2023 20:18 WIB",Manchester Unitedmendapatkan kabar buruk karen...,Olahraga
447,https://www.cnnindonesia.com/olahraga/20231103...,Erick Thohir Ungkap Justin Hubner Selangkah La...,"Jumat, 03 Nov 2023 19:45 WIB",Ketua Umum PSSIErick Thohir mengatakan proses ...,Olahraga
448,https://www.cnnindonesia.com/olahraga/20231103...,"Jelang Korea Masters, Rahmat Tak Merasa Tertek...","Jumat, 03 Nov 2023 19:32 WIB","Pebulutangkis ganda putra Indonesia,Rahmat Hid...",Olahraga


In [ ]:
# Cek jumlah baris dalam dataset
print("Jumlah Baris dalam Dataset:", len(df))

Jumlah Baris dalam Dataset: 450


## 1. Cleaning Data

### Menghapus Data Null

In [ ]:
df.isnull().sum()

Url         0
Judul       0
Tanggal     0
Artikel     0
Category    0
dtype: int64

In [ ]:
df = df.dropna()
df.isnull().sum()

Url         0
Judul       0
Tanggal     0
Artikel     0
Category    0
dtype: int64

In [ ]:
df.shape

(450, 5)

### Menghapus Karakter Tertentu

In [ ]:
def cleaning(text):
  text = re.sub(r'[^\w\s.?!,]', '', text).strip().lower()
  return text

df['Cleaning'] = df['Artikel'].apply(cleaning)
df['Cleaning']

0      pabrikan smartphoneasal china, oppo segera mer...
1      sosok mantan ketua mahkamah konstitusi mk anwa...
2      tampilan user interface ui atau antarmuka baru...
3      ribuan pengguna ramairamai menghapus fitur pem...
4      lubang ozonantartika tahun 2023 mencapai ukura...
                             ...                        
445    kericuhan pecah usai laga pss sleman vs bali u...
446    manchester unitedmendapatkan kabar buruk karen...
447    ketua umum pssierick thohir mengatakan proses ...
448    pebulutangkis ganda putra indonesia,rahmat hid...
449    pemainbali unitednovri setiawan pingsan saat b...
Name: Cleaning, Length: 450, dtype: object

## Stopword Removal

Pada kode tersebut, dilakukan penghapusan kata-kata stopwords dari teks yang telah dibersihkan sebelumnya. Berikut penjelasan singkatnya:


- `corpus = stopwords.words('indonesian')`: Membuat variabel `corpus` yang berisi daftar kata-kata stopwords dalam bahasa Indonesia. Stopwords adalah kata-kata umum yang sering diabaikan dalam analisis teks.
- `def stopwordText(text)`: Membuat fungsi `stopwordText` yang mengambil suatu teks dan mengembalikan teks tersebut setelah melakukan penghapusan kata-kata stopwords.
- `' '.join(word for word in text.split() if word not in corpus)`: Menggabungkan kembali kata-kata yang tidak termasuk dalam stopwords untuk membentuk teks yang sudah melewati proses penghapusan stopwords.
- `df['Stopword Removal'] = df['Cleaning'].apply(stopwordText)`: Menerapkan fungsi `stopwordText` pada kolom 'Cleaning' dari DataFrame `df` dan hasilnya disimpan dalam kolom baru 'Stopword Removal'.

Dengan langkah ini, teks pada kolom 'Stopword Removal' merupakan hasil dari teks yang telah dibersihkan dan telah dihapus stopwords-nya. Data ini dapat digunakan untuk analisis lebih lanjut.

In [ ]:
corpus = stopwords.words('indonesian')

def stopwordText(text):
  return ' '.join(word for word in text.split() if word not in corpus)

df['Stopword Removal'] = df['Cleaning'].apply(stopwordText)

## 2. Tokenizing

Pada kode di atas, fungsi `tokenizer` digunakan untuk melakukan tokenisasi kalimat pada teks yang telah melewati tahap pembersihan dan penghapusan stopwords. Berikut adalah penjelasan singkatnya:

- `text = text.lower()`: Mengubah semua huruf dalam teks menjadi huruf kecil untuk konsistensi.
- `return sent_tokenize(text)`: Menggunakan fungsi `sent_tokenize` dari pustaka NLTK untuk melakukan tokenisasi kalimat.

Selanjutnya, fungsi `tokenizer` diterapkan pada kolom 'Stopword Removal' dari DataFrame `df` menggunakan metode `apply`:

```python
df['Tokenizing'] = df['Stopword Removal'].apply(tokenizer)
df['Tokenizing']
```

Hasil dari tokenisasi kalimat tersebut kemudian disimpan dalam kolom baru 'Tokenizing' pada DataFrame `df`. Kolom ini berisi daftar kalimat-kalimat yang telah dipecah menjadi token-token. Data ini dapat digunakan untuk analisis lebih lanjut, misalnya ekstraksi fitur atau pembentukan model.

In [ ]:
def tokenizer(text):
  text = text.lower()
  return sent_tokenize(text)

df['Tokenizing'] = df['Stopword Removal'].apply(tokenizer)
df['Tokenizing']

0      [pabrikan smartphoneasal china, oppo merilis h...
1      [sosok mantan ketua mahkamah konstitusi mk anw...
2      [tampilan user interface ui antarmuka xiaomi, ...
3      [ribuan pengguna ramairamai menghapus fitur pe...
4      [lubang ozonantartika 2023 mencapai ukuran mak...
                             ...                        
445    [kericuhan pecah laga pss sleman vs bali unite...
446    [manchester unitedmendapatkan kabar buruk case...
447    [ketua pssierick thohir proses naturalisasi pe...
448    [pebulutangkis ganda putra indonesia,rahmat hi...
449    [pemainbali unitednovri setiawan pingsan berta...
Name: Tokenizing, Length: 450, dtype: object

## Fungsi Plot Graph

Fungsi `plot_graph` pada kode tersebut digunakan untuk menggambar graf menggunakan pustaka NetworkX dan Matplotlib. Berikut adalah penjelasan singkatnya:

- `pos = nx.spring_layout(G)`: Menentukan posisi simpul menggunakan layout spring (metode penataan simpul).
- `labels = nx.get_edge_attributes(G, 'weight')`: Mendapatkan atribut berat dari setiap edge (sisi) di graf.
- `plt.figure(figsize=(35, 30))`: Menentukan ukuran canvas untuk gambar graf.
- `nx.draw(G, pos, with_labels=True, node_size=700, node_color='skyblue')`: Menggambar graf dengan node (simpul) berukuran 700, berwarna 'skyblue', dan menampilkan label pada setiap simpul.
- `nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_color='red')`: Menambahkan label pada setiap sisi graf dengan warna teks 'red'.
- `plt.show()`: Menampilkan gambar graf.

Fungsi ini dapat digunakan untuk visualisasi graf yang dibentuk menggunakan pustaka NetworkX.

In [ ]:
def plot_graph(G, figsize=(35, 30), node_size=700, node_color='skyblue'):
  # Menggambar graf dengan canvas yang diperbesar
  pos = nx.spring_layout(G)  # Menentukan posisi simpul
  labels = nx.get_edge_attributes(G, 'weight')

  # Menentukan ukuran canvas
  plt.figure(figsize=(35, 30))

  # Menggambar graf dengan ukuran canvas yang diperbesar
  nx.draw(G, pos, with_labels=True, node_size=700, node_color='skyblue')
  nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_color='red')
  plt.show()

## Fungsi Membangun Graph dengan Co-occurence Matriks

Fungsi `graph_co_occurrence` pada kode tersebut digunakan untuk membuat graf berdasarkan matriks co-occurrence dari teks yang diberikan. Berikut adalah penjelasan singkatnya:

- `vectorizer = CountVectorizer()`: Membuat objek CountVectorizer untuk menghitung frekuensi kemunculan kata-kata.
- `tfidf_matrics = vectorizer.fit_transform(x)`: Mengubah teks menjadi matriks TF-IDF (Term Frequency-Inverse Document Frequency).
- `co_occurrence_matrix = tfidf_matrics.T.dot(tfidf_matrics).toarray()`: Menghitung matriks co-occurrence dari matriks TF-IDF.
- `df_co_occurrence = pd.DataFrame(co_occurrence_matrix, columns=vectorizer.get_feature_names_out())`: Membuat DataFrame dari matriks co-occurrence.
- `if show_matrics: display(df_co_occurrence)`: Jika `show_matrics` bernilai `True`, maka tampilkan matriks co-occurrence.
- `G = nx.DiGraph()`: Membuat objek graf yang merupakan directed graph (graf berarah) menggunakan NetworkX.
- Looping untuk menambahkan edge ke graf berdasarkan matriks co-occurrence dengan bobot di atas ambang batas (`threshold`).
- Mengembalikan objek graf (`G`).

Fungsi ini berguna untuk membentuk graf berdasarkan hubungan co-occurrence antar-kata dalam teks, dengan memanfaatkan matriks co-occurrence.

In [ ]:
def graph_co_occurrence(x, threshold=0, show_matrics = False):
    vectorizer = CountVectorizer()
    tfidf_matrics = vectorizer.fit_transform(x)
    co_occurrence_matrix = tfidf_matrics.T.dot(tfidf_matrics).toarray()
    df_co_occurence = pd.DataFrame(co_occurrence_matrix, columns=vectorizer.get_feature_names_out())

    if show_matrics:
      display(df_co_occurence)

    G = nx.DiGraph()

    # Menambahkan edge ke graf berdasarkan matriks co-occurrence
    for i in range(len(co_occurrence_matrix)):
      for j in range(i + 1, len(co_occurrence_matrix)):
        weight = co_occurrence_matrix[i, j]
        if weight > threshold:
          G.add_edge(vectorizer.get_feature_names_out()[i], vectorizer.get_feature_names_out()[j], weight=weight)
    return G

## Fungsi Ekstrak Kata Teratas

Fungsi `extract_top_words` mengambil teks `x` dan parameter opsional seperti `w` (jumlah kata kunci yang ingin diekstrak), `threshold` (ambang batas untuk co-occurrence), dan beberapa parameter lainnya untuk menyesuaikan output yang dihasilkan. Fungsi ini menggabungkan beberapa langkah pengolahan teks dan pembentukan graf co-occurrence dengan menghitung skor PageRank (TextRank) untuk setiap kata dalam graf. Hasilnya adalah kata-kata kunci yang diurutkan berdasarkan skor tertinggi.

Fungsi ini dapat membantu dalam mengekstrak kata-kata kunci yang paling penting dari teks, dengan mempertimbangkan hubungan co-occurrence antar-kata. Jika parameter `show_scores` diatur sebagai `True`, fungsi ini juga mencetak skor TextRank untuk setiap kata, memberikan pandangan lebih rinci tentang pentingnya masing-masing kata dalam konteks dokumen tersebut.

In [ ]:
def extract_top_words(x, w=3, threshold=0, show_matrics=False, show_scores=False, index=None):
    full_text = ' '.join(word for word in x)

    G = graph_co_occurrence(x, threshold, show_matrics)

    # Menghitung nilai dari PageRank (TextRank)
    scores = nx.pagerank(G)

    # Dictionary untuk menyimpan skor tertinggi setiap kata
    ranked_words_dict = {}

    for word in ' '.join(x).split():
        current_score = scores.get(word, 0)
        if word not in ranked_words_dict or current_score > ranked_words_dict[word]:
            ranked_words_dict[word] = current_score

    # Mengurutkan kata-kata berdasarkan skor tertinggi
    ranked_words = sorted(((score, word) for word, score in ranked_words_dict.items()), key=lambda x: (x[0], x[1]), reverse=True)

    # Memilih sejumlah w kata tertinggi
    selected_words = [word for _, word in sorted(ranked_words[:w], key=lambda x: x[1])] if w is not None else None

    # Menggabungkan kata-kata menjadi satu string terpisah dengan koma
    keywords = ', '.join(selected_words) if selected_words else ''

    if show_scores:
      print(f'Dokumen ke {index} : {full_text}')
      print(f'{w} Kata Kunci : {keywords}')
      print("TextRank Scores:")
      for score, word in ranked_words:
          print(f"Skor: {score}, Kata: {word}")

    return (G, selected_words)

## Fungsi Ekstrak Kata Kunci Ke Semua Berita

Fungsi `extract_all` merupakan fungsi yang membungkus beberapa langkah analisis teks untuk memberikan output yang lebih lengkap. Fungsi ini mengambil teks `x` dan beberapa parameter opsional, seperti `w` (jumlah kata kunci yang ingin diekstrak), `plot` (apakah ingin menampilkan plot graf), `show_matrics` (apakah ingin menampilkan matriks co-occurrence), `show_scores` (apakah ingin menampilkan skor TextRank), `index` (indeks dokumen), dan `threshold` (ambang batas untuk co-occurrence).

Fungsi ini menggunakan fungsi `extract_top_words` untuk mendapatkan kata-kata kunci dan graf co-occurrence. Jika parameter `plot` diatur sebagai `True`, fungsi ini juga memanggil fungsi `plot_graph` untuk menampilkan plot graf.

Hasil akhir dari fungsi ini adalah kata-kata kunci yang dihasilkan dari metode PageRank (TextRank). Jika diperlukan, graf co-occurrence dapat diplot, matriks co-occurrence dapat ditampilkan, dan skor TextRank untuk setiap kata dapat dicetak.

In [ ]:
def extract_all(x, w=4, plot=False, show_matrics=False, show_scores=False, index=1, threshold=0):

  # G = graph_co_occurrence(x, show_matrics)

  G, top_words = extract_top_words(x, w, threshold, show_matrics, show_scores, index)

  if plot:
    print()
    print("Plot Graph :")
    plot_graph(G)

  # Menggabungkan kata-kata menjadi satu string terpisah dengan koma
  keywords = ', '.join(top_words) if top_words else ''

  return keywords

## Contoh Penggunaan 1 Data

### Ambil 1 Data

Dalam contoh ini, kita menggunakan teks tokenisasi dari baris kedua (iloc[1]) dalam kolom 'Tokenizing' dari DataFrame df. Fungsi extract_all kemudian akan memberikan kata-kata kunci berdasarkan langkah-langkah analisis teks yang telah diimplementasikan sebelumnya. Jika plot diatur sebagai True, akan ditampilkan plot graf co-occurrence, jika show_matrics diatur sebagai True, akan ditampilkan matriks co-occurrence, dan jika show_scores diatur sebagai True, akan dicetak skor TextRank untuk setiap kata.

In [ ]:
example = df["Tokenizing"].iloc[1]
example

['sosok mantan ketua mahkamah konstitusi mk anwar usmanjadi sorotan warganet dinyatakan melanggar kode etik berat.',
 'anwar dijatuhi sanksi pencopotan jabatannya ketua mk.',
 'majelis kehormatan mahkamah konstitusi mkmk sehari membacakan putusan terkait pelanggaran kode etik anwar putusan mk syarat usia caprescawapres selasa 711. kemarin, warganet platform x twitter riuh membahas sosok anwar usman.',
 'menyinggung majunya gibran rakabuming raka cawapres putusan bermasalah.',
 'scroll to continue with content gibran_tweet sampean cawapres dasar pelanggaran berat penegak hukum, ketua mk, om bapak.',
 'ra isin, akun saiful_mujani, selasa 711. gambastwitter itu, akun chicohakim menilai putusan mkmk anwar usman keputusan tanggung.',
 'pasalnya, anwar dicopot ketua mk, jabatan hakim mk.',
 'putusan mkmk mencopot anwar usman jabatannya ketua mk, memberhentikannya jabatan hakim mk sesungguhnya keputusan tanggung upaya menebus dosa putusan mk syarat caprescawapres.',
 'ya sudahlah, cuitnya.',


### Ekstrak Keywoards

Dengan ini, keyword akan berisi kata kunci yang dihasilkan dari teks example berdasarkan parameter-parameter yang telah diatur. Jika show_matrics diatur sebagai True, matriks co-occurrence akan ditampilkan. Jika plot diatur sebagai True, graf co-occurrence akan diplot. Dan jika show_scores diatur sebagai True, skor TextRank untuk setiap kata akan dicetak.

In [ ]:
keyword =  extract_all(example, w=3, show_matrics=True, plot=True, show_scores=True, index=1)

Output hidden; open in https://colab.research.google.com to view.

### Keywoards Yang di hasilkan

In [ ]:
keyword

'usman, warganet, yg'

## Ekstrak Keyword di semua Berita

Dalam fungsi apply, row["Tokenizing"] merujuk pada teks tokenisasi di setiap baris, dan row.name + 1 merujuk pada indeks baris (ditambah 1 untuk membuat indeks dimulai dari 1).

Kolom baru "Kata Kunci" akan berisi kata kunci yang dihasilkan dari setiap baris berdasarkan fungsi extract_all. Jika show_scores diatur sebagai True, skor TextRank untuk setiap kata akan dicetak.

In [ ]:
df["Kata Kunci"] = df.apply(lambda row: extract_all(row["Tokenizing"], w=3, show_scores=True, index=row.name+1), axis=1)

Streaming output truncated to the last 5000 lines.
Skor: 0.017726545009628312, Kata: sabtu
Skor: 0.01765023734494254, Kata: rela
Skor: 0.01640517367600617, Kata: sepenuhnya
Skor: 0.016362892906650547, Kata: spekulasi
Skor: 0.01461959153313299, Kata: ramai
Skor: 0.014588597353096379, Kata: madrid
Skor: 0.014519298737570416, Kata: striker
Skor: 0.014374752262761635, Kata: transfer
Skor: 0.014027063231700908, Kata: musim
Skor: 0.013421241712115551, Kata: to
Skor: 0.013320491579994365, Kata: salah
Skor: 0.01325335444825515, Kata: setara
Skor: 0.012495321089735239, Kata: raksasa
Skor: 0.011694893665898836, Kata: sportmenjadi
Skor: 0.011410253931464366, Kata: media
Skor: 0.009540836933626541, Kata: pindah
Skor: 0.00944155073279469, Kata: prancis
Skor: 0.009418744064484726, Kata: scroll
Skor: 0.0093764350082448, Kata: posisi
Skor: 0.009263571236757792, Kata: pemain
Skor: 0.008999756887000044, Kata: paris
Skor: 0.008845183908360319, Kata: real
Skor: 0.007791382614733042, Kata: perpanjangan
Sko

## Membuat DataFrame Baru dengan 3 Kolom (Artikel, Kata Kunci, dan Category)

In [ ]:
df_KataKunci = df[['Artikel', 'Kata Kunci', 'Category']]
df_KataKunci

,Artikel,Kata Kunci,Category
0,"Pabrikan smartphoneasal China, Oppo segera mer...","utama, wide, zoom",Teknologi
1,Sosok mantan Ketua Mahkamah Konstitusi (MK) An...,"usman, warganet, yg",Teknologi
2,Tampilan user interface (UI) atau antarmuka ba...,"sistem, tampilan, xiaomi",Teknologi
3,Ribuan pengguna ramai-ramai menghapus fitur pe...,"perusahaan, video, youtube",Teknologi
4,Lubang ozonAntartika tahun 2023 mencapai ukura...,"ozon, september, watch",Teknologi
...,...,...,...
445,Kericuhan pecah usai laga PSS Sleman vs Bali U...,"suporter, united, vs",Olahraga
446,Manchester Unitedmendapatkan kabar buruk karen...,"tim, united, zona",Olahraga
447,Ketua Umum PSSIErick Thohir mengatakan proses ...,"u20, william, wolves",Olahraga
448,"Pebulutangkis ganda putra Indonesia,Rahmat Hid...","rahmat, tekanan, yongki",Olahraga
